## Introduction to Evolutionary Algorithm

In the field of computer science and operation reseach, a genetic algorithm (GA) is a metaheuristic optimization method inspired by natural selection processes. It belongs to a larger class of evolutionary algorithms (EA) and is the most widely known one. Evolutionary Algorithm comes into picture when we would like to find an optimal solution for a given problem.

Evolutionary algorithms have three main characteristics as oppsed to traditional algorithms: 

1. **`Population-Based`**: Evolutionary algorithms are optimized through population -- which is the set of current solutions. New solutions are generated from this population through breeding and mutations.

2. **`Fitness-Oriented`**: As we need loss functions to determine the best results, evolutionary algorithms utilize **Fitness** to evaluate how good the solution is.


3. **`Variation-Driven`**: As the size of the problem grows, it's impossible to cover all the possibilities in population. Generally speaking, we can generate a population that contains some portion of the entire feasible solutions. If there is no better solution throughout the current population, Evolutionary Algorithms are able to breed and mutate and generate new solutions that might be better than all in the current population. Namely, Evolutionary Algorithms are driven by mutation to generate new better solutions.

## Genetic Algorithm

Genetic Algorithm is random-based evolutionary algorithm, which means that random changes are applied to current solutions to generate new solutions. It is also called Bionics Genetic Algorithm, as it is a learned evolution process from looking at various natural species as they evolve. Genetic Algorithm has its core idea from **`Charles Darwin's theory of natural evolution -- "survival of the fittest"`**. Better genes are kept along through a series of breeding and mutations. It is a slow and gradual process that makes slight changes to current solutions. After a number of epochs, we can get a better result compared to the initial one.

Genetic Algorithm is widely used in the world of data science. For example, genetic algorithm can be used to optimize the structure of artifical neural network. Meanwhile, it is highly used in image sementation and enhancement. 

In general, Genetic Algorithm includes the following steps:

<img src="pic/flow.png">

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import operator
import datetime
import random

In [2]:
class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((xDis**2) + (yDis**2))
        return(distance)
    
    def __repr__(self):
        return("(" + str(self.x) + "," + str(self.y) + ")")

In [3]:
class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = 0.0
        self.fitness = 0.0
    
    def routeDistance(self):
        if self.distance == 0.0:
            pathDistance = 0
            for i in range(0, len(self.route)):
                fromCity = self.route[i]
                toCity = None
                if i + 1 < len(self.route):
                    toCity = self.route[i+1]
                else:
                    toCity = self.route[0]
                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return(self.distance)
    
    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.routeDistance()) # fitness is the reciprocal of routeDistance
        return(self.fitness)

In [4]:
def createRoute(cityList):
    route = random.sample(cityList, len(cityList))
    return(route)

In [5]:
def initialPopulation(popSize, cityList):
    population = []
    for i in range(0, popSize):
        population.append(createRoute(cityList))
    return(population)

In [6]:
def rankRoutes(population):
    fitnessResults = {}
    for i in range(0, len(population)):
        fitnessResults[i] = Fitness(population[i]).routeFitness()
    return(sorted(fitnessResults.items(), key=operator.itemgetter(1), reverse=True))

In [7]:
def selection(popRanked, eliteSize):
    selectionResults = []
    df = pd.DataFrame(np.array(popRanked), columns=['index','Fitness'])
    df['cum_sum'] = df.Fitness.cumsum()
    df['cum_perc'] = 100 * df.cum_sum / df.Fitness.sum()
    
    for i in range(0, eliteSize):
        selectionResults.append(popRanked[i][0]) # store the elites
    for i in range(0, len(popRanked) - eliteSize): # Randomly sample other population
        pick = 100 * random.random()
        for i in range(0, len(popRanked)):
            if pick <= df.iat[i, 3]:
                selectionResults.append(popRankedp[i][0])
                break
    return(selectionResults)

In [8]:
def matingPool(population, selectionResults):
    matingPool = []
    for i in range(0, len(selectionResults)):
        index = selectionResults[i]
        matingPool.append(population[i])
    return(matingPool)

In [10]:
def breed(parent1, parent2):
    child = []
    childP1 = []
    childP2 = []
    
    geneA = int(random.random() * len(parent1)) # Randomly select genes
    geneB = int(random.random() * len(parent1))
    
    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)
    
    for i in range(startGene, endGene):
        childP1.append(parent1[i])
    
    childP2 = [item for item in parent2 if item not in childP1]
    
    child = childP1 + childP2
    return(child)

In [32]:
def breedPopulation(matingPool, eliteSize):
    children = []
    length = len(matingPool) - eliteSize
    pool = random.sample(matingPool, len(matingPool)) # shuffle the matingPool
    
    for i in range(0, eliteSize):
        children.append(matingPool[i])
    
    for i in range(0, length):
        child = breed(pool[i], pool[len(matingPool) - i - 1])
        children.append(child)
    return(children)

In [33]:
def mutate(individual, mutationRate):
    for swapped in range(len(individual)):
        if random.random() < mutationRate:
            swapWith = int(random.random() * len(individual))
            city1 = individual[swapped]
            city2 = individual[swapWith]
            
            individual[swapped] = city2
            individual[swapWith] = city1
    return(individual)

In [34]:
def mutatePolulation(population, mutationRate):
    mutatedPop = []
    
    for ind in range(0, len(population)):
        mutatedInd = mutate(population[ind], mutationRate)
        mutatedPop.append(mutatedInd)
    return(mutatedPop)

In [35]:
def nextGeneration(currentGen, eliteSize, mutationRate):
    popRanked = rankRoutes(currentGen)
    selectionResults = selection(popRanked, eliteSize)
    matingpool = matingPool(currentGen, selectionResults)
    children = breedPopulation(matingpool, eliteSize)
    nextGen = mutatePolulation(children, mutationRate)
    
    return(nextGen)

In [36]:
def geneticAlgorithm(population, popSize, eliteSize, mutationRate, generations):
    pop = initialPopulation(popSize, population)
    print("Initial distance: " + str(1 / rankRoutes(pop)[0][1]))
    
    for i in range(0, generations):
        pop = nextGeneration(pop, eliteSize, mutationRate)
    
    obj = 1 / rankRoutes(pop)[0][1]
    print("Final distance: " + str(obj))
    bestRouteIndex = rankRoutes(pop)[0][1]
    bestRoute = pop[bestRouteIndex]
    return(bestRoute, obj)

In [37]:
def geneticAlgorithmPlot(population, popSize, eliteSize, mutationRate, generations):
    pop = initalPopulation(popSize, population)
    progress = []
    progress.append(1 / rankRoutes(pop)[0][1])
    
    for i in range(0, generations):
        start = datetime.datetime.now()
        pop = nextGeneration(pop, eliteSize, mutationRate)
        progress.append(1/rankRoutes(pop)[0][1])
#         if len(progess) % 50 == 0:
#             print(f"{100 * len(progess) / generations}% Sucess. Time spent: {datetime.datetime.now() - start}.")
    
    plt.plot(progess)
    plt.ylabel('Distance')
    plt.xlabel('Generation')
    plt.show()

---

## Reference: 

https://en.wikipedia.org/wiki/Genetic_algorithm

https://www.it4nextgen.com/genetic-algorithm/

https://towardsdatascience.com/introduction-to-optimization-with-genetic-algorithm-2f5001d9964b